<a href="https://colab.research.google.com/github/itsjafer/decktor-cli/blob/main/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeckTor Quickstart

Improve your Anki decks using Google's Gemini API directly in your browser. No coding required!


> **⚠️ IMPORTANT:** The default mode works best with basic **Front/Back** cards. If your cards have many fields or complex templates, you must provide a custom prompt using the CLI options below.

In [4]:
#@title 1. Install & Setup
#@markdown Run this cell to install DeckTor and configure your API key.

import os
from IPython.display import clear_output

# Install package
!pip install git+https://github.com/itsjafer/decktor-cli.git
clear_output()
print("✅ Installation complete!")

# API Key Setup
print("\n--- Optional: Gemini API Key ---")
print("If you want to use official Gemini models (faster/more stable), enter your key below.")
print("If you want to use the GPT4Free (Free), just press Enter or leave the field empty.")
GOOGLE_API_KEY = "Enter your key here (or leave blank for Free model)" #@param {type:"string"}

if GOOGLE_API_KEY and GOOGLE_API_KEY != "Enter your key here (or leave blank for Free model)":
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ API Key configured!")
else:
    print("⚠️  No API Key provided. You can only use 'GPT4Free (Free)'.")


✅ Installation complete!

--- Optional: Gemini API Key ---
If you want to use official Gemini models (faster/more stable), enter your key below.
If you want to use the GPT4Free (Free), just press Enter or leave the field empty.
⚠️  No API Key provided. You can only use 'GPT4Free (Free)'.


In [5]:
#@title 2. Upload Your Deck
#@markdown Click the play button to upload your `.apkg` file. **Ensure your deck uses basic Front/Back cards unless using a custom prompt.**

from google.colab import files

uploaded = files.upload()
input_filename = next(iter(uploaded))
print(f"\n✅ Uploaded: {input_filename}")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#@title 3. Run DeckTor
#@markdown Choose your model and start processing. By default, only the first 50 cards will be processed, 10 at a time.

output_filename = "improved_" + input_filename

# Configuration Options
model = "Gemini 2.5 Flash Lite" #@param ["Gemini 2.5 Flash Lite", "Gemini 2.5 Flash", "Gemini 2.0 Flash", "GPT4Free (Free)"]
batch_size = 10 #@param {type:"integer"}
limit = 10 #@param {type:"integer"}

# Auto-switch to Free model if no API Key is set
import os
if not os.environ.get("GOOGLE_API_KEY"):
    if model != "GPT4Free (Free)":
        print("ℹ️ No API Key found. Switching to GPT4Free (Free) model.")
        model = "GPT4Free (Free)"


# Construct command
cmd = f'decktor process "{input_filename}" "{output_filename}" --model "{model}" --batch-size {batch_size}'

if limit > 0:
    cmd += f' --limit {limit}'

print(f"Running: {cmd}\n")
!{cmd}

print(f"\n✅ Done! Saved to: {output_filename}")

In [ ]:
#@title 4. Download Result
from google.colab import files
files.download(output_filename)